In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#load the datasets

socioeconomic_df = pd.read_csv("socioeconomic_dataset.csv")
health_burden_df = pd.read_csv("health_burden_dataset.csv")
risk_factors_df = pd.read_csv("risk_factor_dataset.csv")

In [4]:
risk_factors_df.head()

,measure,location,cause,risk_factor,metric,year,val
0,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2000,257.840865
1,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2001,249.595886
2,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2002,245.426604
3,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2003,268.556783
4,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2004,298.192621


In [5]:
# Standardize column names for consistency
socioeconomic_df.rename(columns={'Country': 'country', 'Year': 'year'}, inplace=True)
health_burden_df.rename(columns={'location': 'country'}, inplace=True)
risk_factors_df.rename(columns={'location': 'country'}, inplace=True)

In [6]:
# Step 2: Normalize text values for merging
socioeconomic_df['country'] = socioeconomic_df['country'].str.strip().str.lower()
health_burden_df['country'] = health_burden_df['country'].str.strip().str.lower()
risk_factors_df['country'] = risk_factors_df['country'].str.strip().str.lower()

In [7]:
# Step 3: Subset COPD-specific rows from burden and risk datasets
copd_burden_df = health_burden_df[
    health_burden_df['cause'].str.contains("chronic obstructive pulmonary disease", case=False)
].copy()

copd_risk_df = risk_factors_df[
    risk_factors_df['cause'].str.contains("chronic obstructive pulmonary disease", case=False)
].copy()

In [8]:
# Step 4: Pivot health burden data to wide format ---
burden_pivot = copd_burden_df.pivot_table(
    index=['country', 'year'],
    columns='measure',
    values='val',
    aggfunc='mean'
).reset_index()

In [9]:
# Step 5: Pivot risk factor data to wide format ---
risk_pivot = copd_risk_df.pivot_table(
    index=['country', 'year'],
    columns='risk_factor',
    values='val',
    aggfunc='mean'
).reset_index() 

In [10]:
# Step 6: Merge all datasets on 'country' and 'year' 
merged_df = socioeconomic_df.merge(burden_pivot, on=['country', 'year'], how='left')
merged_df = merged_df.merge(risk_pivot, on=['country', 'year'], how='left')

# Preview the final merged dataset
print("Merged DataFrame shape:", merged_df.shape)
print(merged_df.head())


Merged DataFrame shape: (1040, 28)
   country       Sub-Region  year  Population  GDP PER CAPITA (USD)  \
0  algeria  Northern Africa  2000    30774621               1780.38   
1  algeria  Northern Africa  2001    31200985               1754.58   
2  algeria  Northern Africa  2002    31624696               1794.81   
3  algeria  Northern Africa  2003    32055883               2117.05   
4  algeria  Northern Africa  2004    32510186               2624.80   

   Area (Km2)  Population Density  Total CO2 Emission excluding LUCF (Mt)  \
0     2381741           12.921061                                   80.05   
1     2381741           13.100075                                   78.65   
2     2381741           13.277974                                   82.40   
3     2381741           13.459013                                   88.19   
4     2381741           13.649757                                   89.49   

   Nitrogen Oxide  Sulphur Dioxide  ...       Deaths     Incidence  \
0    